In [1]:
import numpy as np
import tensorflow as tf
import time

In [2]:
# N observations with D parameters
N = 2000
D = 50
# projection dimension
K = 400
# variance parameter
sigma = 3
# Y = Xb + N(0, sigma)
beta_true = np.random.uniform(-10, 10, D).reshape(D, 1)
dataX = np.random.normal(0, 1, N * D).reshape(N, D)
dataY = np.random.normal(0, sigma, N).reshape(N, 1) + np.dot(dataX, beta_true)

In [3]:
beta_LS = np.dot(np.dot(np.linalg.inv(np.dot(np.transpose(dataX), dataX)), np.transpose(dataX)), dataY)

In [4]:
# placeholding tensors and variable
X = tf.placeholder('float', [None, D]) 
Y = tf.placeholder('float', [None, 1.0])
beta = tf.Variable(tf.random_normal([D, 1], stddev=1.0))

In [5]:
# linear regression with mean squared error
Y_hat = tf.matmul(X, beta)
MSE = tf.reduce_sum(tf.square(Y - Y_hat))

In [6]:
# gradient
grad = tf.gradients(MSE, beta)

In [7]:
# hessian (or use tf.hessians in Tensorflow 1.0)
def compute_hessian():
    for i in range(D):
        # element in the gradient vector
        dfdx_i = tf.slice(grad[0], begin=[i, 0], size=[1, 1])
        # differentiate again
        ddfdx2_i = tf.gradients(dfdx_i, beta)[0]
        # combine second derivative vectors
        if i == 0:
            hess = ddfdx2_i
        else:
            hess = tf.concat(1, [hess, ddfdx2_i])
    return(hess)

hessian = compute_hessian()

In [8]:
# fisher information
fisher = tf.matrix_inverse(hessian)

In [9]:
# update beta by delta
delta = tf.placeholder('float', [D, 1])
drop = beta.assign_add(delta)

In [10]:
beta_sketch = np.zeros((D, 1))

with tf.Session() as sess:
    tf.initialize_all_variables().run()
    for i in range(0, 10):
        # gaussian random projection
        S = np.random.normal(0, 1, K * N).reshape(K, N)
        SX = np.dot(S, dataX)
        SY = np.dot(S, dataY)
        # compute gradient
        g = sess.run(grad, feed_dict={X: SX, Y: SY})[0]
        # compute hessian
        I = sess.run(fisher, feed_dict={X: SX, Y: SY})
        # drop
        sess.run(drop, feed_dict={delta : -np.dot(I, g)})
    beta_sketch = beta.eval()

In [11]:
ratio = np.mean(np.square(beta_true - beta_sketch)) / np.mean(np.square(beta_true - beta_LS))
print(ratio)

6.62398758447
